In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
conn = sqlite3.connect(r"c:\Dev\06. Data\3. FDA NCTs - SQL\studies_db_06-Nov-2018_all date fields as ints.sqlite3")
df = pd.read_sql_query("SELECT * FROM location_countries", conn)
df.head()

,id,nct_id,country
0,1,NCT00000102,United States of America
1,2,NCT00000104,United States of America
2,3,NCT00000105,United States of America
3,4,NCT00000106,United States of America
4,5,NCT00000107,United States of America


In [3]:
df_countries_list = pd.read_sql_query("SELECT DISTINCT country FROM location_countries ORDER BY country ASC", conn)
df_countries_list.to_csv(r"c:\Dev\06. Data\3. FDA NCTs - SQL\countries_from_sql.csv")
df_countries_list.head()

,country
0,**None**
1,Afghanistan
2,Albania
3,Algeria
4,American Samoa


In [4]:
country_phase_by_year = pd.read_sql_query("""
    SELECT Count(s1.nct_id) as nct_id, strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch')) as year
            FROM studies as s1
            WHERE s1.phase = 'Phase 1' or s1.phase = 'Early Phase 1'
            GROUP BY strftime('%Y', datetime(s1.study_first_submitted, 'unixepoch'))  
            ORDER BY year ASC
""", conn)
country_phase_by_year.head()

,nct_id,year
0,684,1999
1,255,2000
2,231,2001
3,181,2002
4,233,2003


In [5]:
country_phase_by_year.index = country_phase_by_year.year
country_phase_by_year.drop(columns=["year"], axis=1, inplace=True)
country_phase_by_year.head()

,nct_id
year,
1999,684
2000,255
2001,231
2002,181
2003,233


In [6]:
print(country_phase_by_year.to_dict())

{'nct_id': {'1999': 684, '2000': 255, '2001': 231, '2002': 181, '2003': 233, '2004': 247, '2005': 844, '2006': 1011, '2007': 1514, '2008': 2179, '2009': 2460, '2010': 2300, '2011': 2398, '2012': 2329, '2013': 2218, '2014': 3019, '2015': 2437, '2016': 2493, '2017': 2418, '2018': 2015}}


In [9]:
conn.execute("""
    CREATE VIEW IF NOT EXISTS counted_countries AS 
        SELECT country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) as year, 
                COUNT(nct_id) as id_count 
        FROM (
            SELECT location_countries.country as country, stud.* 
            FROM studies as stud 
        LEFT JOIN location_countries ON location_countries.nct_id = stud.nct_id ) as s
        GROUP BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) 
        ORDER BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
""")

OperationalError: database is locked

In [18]:
df_studies_phases_by_year = pd.read_sql_query("""
    SELECT counted_countries.country, counted_countries.year, COALESCE(SUM(phase1), 0) as phase1, 
        COALESCE(SUM(phase2), 0) as phase2, COALESCE(SUM(phase3), 0) as phase3, 
        COALESCE(SUM(phase4), 0) as phase4,
        COALESCE(SUM(phase1), 0)+COALESCE(SUM(phase2), 0)+COALESCE(SUM(phase3), 0)+COALESCE(SUM(phase4), 0) as Total
FROM counted_countries 
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase1 
        FROM counted_countries 
        WHERE phase = "Phase 1" GROUP BY country, phase, year ) as phase1Tbl 
    ON counted_countries.country = phase1Tbl.country and counted_countries.phase = phase1Tbl.phase and counted_countries.year = phase1Tbl.year 
    
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase2 
        FROM counted_countries 
        WHERE phase = "Phase 2" GROUP BY country, phase, year ) as phase2Tbl 
    ON counted_countries.country = phase2Tbl.country and counted_countries.phase = phase2Tbl.phase and counted_countries.year = phase2Tbl.year 
    
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase3 
        FROM counted_countries 
        WHERE phase = "Phase 3" GROUP BY country, phase, year ) as phase3Tbl 
    ON counted_countries.country = phase3Tbl.country and counted_countries.phase = phase3Tbl.phase and counted_countries.year = phase3Tbl.year 
    
    LEFT JOIN ( 
        SELECT country, phase, year, SUM(id_count) as phase4 
        FROM counted_countries 
        WHERE phase = "Phase 4" GROUP BY country, phase, year ) as phase4Tbl 
    ON counted_countries.country = phase4Tbl.country and counted_countries.phase = phase4Tbl.phase and counted_countries.year = phase4Tbl.year

	GROUP BY counted_countries.country, counted_countries.year
    """, conn)
df_studies_phases_by_year.head()

,country,year,phase1,phase2,phase3,phase4,Total
0,**None**,1999,18,63,172,2,255
1,**None**,2000,19,48,41,1,109
2,**None**,2001,10,13,3,1,27
3,**None**,2002,4,42,65,12,123
4,**None**,2003,10,36,28,2,76


In [19]:
df_studies_phases_by_year.loc[df_studies_phases_by_year.country=="Ukraine"].head()

,country,year,phase1,phase2,phase3,phase4,Total
2369,Ukraine,2001,0,0,1,0,1
2370,Ukraine,2002,0,0,5,0,5
2371,Ukraine,2003,0,0,6,0,6
2372,Ukraine,2004,0,1,11,0,12
2373,Ukraine,2005,0,15,47,5,67


In [24]:
country_codesDF = pd.read_csv(r"c:\Dev\05. Jupyter\2. FDA countries clean\csv\countries_codes_6Nov.csv")
country_codesDF.head()

,Unnamed: 0,iso2,full_country_name,comment,wiki_link,population,area,flag
0,0,no_code_**none**,**None**,NaN,NaN,-1.0,-1.0,-1
1,1,af,Afghanistan,NaN,https://en.wikipedia.org/wiki/Afghanistan,34656032.0,652864.0,//upload.wikimedia.org/wikipedia/commons/thumb...
2,2,al,Albania,NaN,https://en.wikipedia.org/wiki/Albania,2876591.0,28748.0,//upload.wikimedia.org/wikipedia/commons/thumb...
3,3,dz,Algeria,NaN,https://en.wikipedia.org/wiki/Algeria,42200000.0,2381741.0,//upload.wikimedia.org/wikipedia/commons/thumb...
4,4,as,American Samoa,NaN,https://en.wikipedia.org/wiki/American_Samoa,55689.0,199.0,//upload.wikimedia.org/wikipedia/commons/thumb...


In [35]:
country_codesDF.loc[country_codesDF.full_country_name == "Argentina", "wiki_link"].values[0]

'https://en.wikipedia.org/wiki/Argentina'

In [37]:
import json
BASE_DIR_FOR_FILES = "c:\\Dev\\05. Jupyter\\2. FDA countries clean\\" 
for curr_country in df_countries_list.country:
    print("Parsing country:{}".format(curr_country))
    df_studies_phases_by_year2 = pd.DataFrame(df_studies_phases_by_year.loc[df_studies_phases_by_year.country==curr_country])
    df_studies_phases_by_year2.index = df_studies_phases_by_year2.year
    df_studies_phases_by_year2 = df_studies_phases_by_year2.drop(columns=["country", "year"])
    curr_country_dict = {}
    curr_country_dict["trials_by_year_and_phase"] = df_studies_phases_by_year2.to_dict(orient="index")
    curr_country_dict["full_country_name"] = country_codesDF.loc[country_codesDF.full_country_name == "Argentina", "full_country_name"].values[0]
    curr_country_dict["flag_link"] = country_codesDF.loc[country_codesDF.full_country_name == "Argentina", "flag"].values[0]
    curr_country_dict["wiki_link"] = country_codesDF.loc[country_codesDF.full_country_name == "Argentina", "wiki_link"].values[0]
    
    
    curr_country_fn = curr_country.replace("*", "_") 
    json.dump(curr_country_dict, open(BASE_DIR_FOR_FILES+curr_country_fn+".json", "w"))


Parsing country:**None**
Parsing country:Afghanistan
Parsing country:Albania
Parsing country:Algeria
Parsing country:American Samoa
Parsing country:Andorra
Parsing country:Angola
Parsing country:Argentina
Parsing country:Armenia
Parsing country:Aruba
Parsing country:Australia
Parsing country:Austria
Parsing country:Azerbaijan
Parsing country:Bahamas
Parsing country:Bahrain
Parsing country:Bangladesh
Parsing country:Barbados
Parsing country:Belarus
Parsing country:Belgium
Parsing country:Belize
Parsing country:Benin
Parsing country:Bermuda
Parsing country:Bhutan
Parsing country:Bolivia
Parsing country:Bosnia and Herzegovina
Parsing country:Botswana
Parsing country:Brazil
Parsing country:Brunei Darussalam
Parsing country:Bulgaria
Parsing country:Burkina Faso
Parsing country:Burundi
Parsing country:Cambodia
Parsing country:Cameroon
Parsing country:Canada
Parsing country:Cayman Islands
Parsing country:Central African Republic
Parsing country:Chad
Parsing country:Chile
Parsing country:China

In [17]:
df_studies_phases_by_year.head()

,country,year,phase1,phase2,phase3,phase4,Total
year,,,,,,,
1999,**None**,1999,18,63,172,2,255
2000,**None**,2000,19,48,41,1,109
2001,**None**,2001,10,13,3,1,27
2002,**None**,2002,4,42,65,12,123
2003,**None**,2003,10,36,28,2,76


In [ ]:
df_studies_phases_by_year.index = df_studies_phases_by_year.month
print(df_studies_phases_by_year.to_dict(orient="index"))

In [ ]:
PHASES_DICT = {
    "unknown" : ["**None**", "N/A"],
    "phase1": ["Phase 1", "Early Phase 1"],
    "phase2": ["Phase 2", "Phase 1/Phase 2"],
    "phase3": ["Phase 3", "Phase 2/Phase 3"],
    "phase4": ["Phase 4"]
}
cond_preamb = "s1.phase = '"
cond_concat = " or "
for country in df_countries_list.country:
    print("Country:{}".format(country))
    for phase in PHASES_DICT:
        where_condition = ""
        phase_condition_array = PHASES_DICT[phase]
        i=0
        while True:
            where_condition += cond_preamb+phase_condition_array[i]+"'"
            i+=1
            if i<len(phase_condition_array):
                where_condition += cond_concat
                continue
            else: break
        print("Phase:{}    Condition:{}".format(phase, where_condition))
        
    

Returns all records (phase, year by country)

SELECT phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) as year, COUNT(nct_id)
    FROM 
	(
		SELECT location_countries.country as country, stud.*
		FROM studies as  stud
		LEFT JOIN location_countries ON location_countries.nct_id = stud.nct_id
		WHERE location_countries.country = "Argentina"
	)
	as s
 GROUP BY phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
 ORDER BY phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
 


view

CREATE VIEW counted_countries
AS
SELECT country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch')) as year, COUNT(nct_id) as id_count
		FROM 
		(
			SELECT location_countries.country as country, stud.*
			FROM studies as  stud
			LEFT JOIN location_countries ON location_countries.nct_id = stud.nct_id
		)
		as s
	 GROUP BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))
	 ORDER BY country, phase, strftime('%Y', datetime(s.study_first_submitted, 'unixepoch'))

GOOD SQL to build table

SELECT counted_countries.country, counted_countries.year, 
		COALESCE(SUM(phase1), 0) as phase1, COALESCE(SUM(phase2), 0) as phase2, 
		COALESCE(SUM(phase3), 0) as phase3, COALESCE(SUM(phase4), 0) as phase4
FROM counted_countries
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase1
	FROM counted_countries
	WHERE phase = "Phase 1"
	GROUP BY country, phase, year
) as phase1Tbl
  ON counted_countries.country = phase1Tbl.country and counted_countries.phase = phase1Tbl.phase and counted_countries.year = phase1Tbl.year
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase2
	FROM counted_countries
	WHERE phase = "Phase 2"
	GROUP BY country, phase, year
) as phase2Tbl
  ON counted_countries.country = phase2Tbl.country and counted_countries.phase = phase2Tbl.phase and counted_countries.year = phase2Tbl.year
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase3
	FROM counted_countries
	WHERE phase = "Phase 3"
	GROUP BY country, phase, year
) as phase3Tbl
  ON counted_countries.country = phase3Tbl.country and counted_countries.phase = phase3Tbl.phase and counted_countries.year = phase3Tbl.year
LEFT JOIN (
	SELECT country, phase, year, SUM(id_count) as phase4
	FROM counted_countries
	WHERE phase = "Phase 4"
	GROUP BY country, phase, year
) as phase4Tbl
  ON counted_countries.country = phase4Tbl.country and counted_countries.phase = phase4Tbl.phase and counted_countries.year = phase4Tbl.year

  
WHERE counted_countries.country = "Argentina"
GROUP BY counted_countries.country, counted_countries.year